In [1]:
%env WANDB_PROJECT=aspect_explain_attack_vector

env: WANDB_PROJECT=aspect_explain_attack_vector


In [2]:
from project_dataset import load_dataset


In [3]:
from dataclasses import dataclass

@dataclass
class Args:
    model_name = "Salesforce/codet5p-220m"
    num_proc = 4
    batch_size = 5
    max_src_length = 1200
    max_des_length = 146
    data_cols = ["CVE ID", "explain", "func_before"]
    save_dir = 'tf_board'
    epochs = 100
    grad_acc_steps = 4
    lr = 5e-5
    log_freq = 10
    local_rank = -1
    deepspeed = None
    fp16 = False
    lr_warmup_steps = 200
    weight_decay = 0.05
    task = "attack_vector"
    
args = Args()

In [4]:
ds = load_dataset(args.task)

In [5]:
import os

os.makedirs(args.save_dir, exist_ok=True)

In [6]:
from transformers import AutoTokenizer
codet5p_tokenizer = AutoTokenizer.from_pretrained(args.model_name)

In [7]:
import numpy as np

def preprocess_function(examples):
    source = [' '.join(ex) for ex in examples["func_before"]]
    target = [' '.join(ex) for ex in examples["explain"]]

    input_feature = codet5p_tokenizer(source, max_length=args.max_src_length, padding="max_length", truncation=True)
    labels = codet5p_tokenizer(target, max_length=args.max_des_length, padding="max_length", truncation=True)

    lables = labels["input_ids"].copy()
    # lables = np.where(lables != codet5p_tokenizer.pad_token_id, lables, -100)

    return {  "input_ids": input_feature["input_ids"],
              "attention_mask": input_feature["attention_mask"],
              "labels": lables}


tokenized_ds = ds.map(
  preprocess_function,
  remove_columns=args.data_cols,
  batched=True,
  num_proc=args.num_proc,
  batch_size=args.batch_size)

tokenized_ds

Map (num_proc=4):   0%|          | 0/4858 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1350 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4858
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 540
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1350
    })
})

In [8]:
import torch
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name).to(device)

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
  codet5p_tokenizer,
  model=model,
  return_tensors="pt")

In [10]:
import evaluate
import numpy as np

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = codet5p_tokenizer(arg)
  return codet5p_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, codet5p_tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = codet5p_tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = codet5p_tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  text_preds = ["\n".join(p for p in text_preds)]
  text_labels = ["\n".join(l for l in text_labels)]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [11]:
# for testing, show first 5 rows

from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)

for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      max_length=args.max_des_length,
    ).cpu()
  labels = batch["labels"].cpu()
  break

metrics_func([preds, labels])

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Token indices sequence length is longer than the specified maximum sequence length for this model (728 > 512). Running this sequence through the model will result in indexing errors


{'rouge1': 0.29867256637168144,
 'rouge2': 0.07317073170731708,
 'rougeL': 0.14823008849557523,
 'rougeLsum': 0.2563559322033898}

In [12]:
preds.shape, labels.shape

(torch.Size([5, 146]), torch.Size([5, 146]))

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
        report_to='wandb',
        output_dir=args.save_dir,
        overwrite_output_dir=False,

        do_train=True,
        save_strategy='epoch',
        do_eval=True,

        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.batch_size,
        gradient_accumulation_steps=args.grad_acc_steps,

        learning_rate=args.lr,
        weight_decay=args.weight_decay,
        warmup_steps=args.lr_warmup_steps,

        logging_dir=args.save_dir,
        logging_first_step=True,
        logging_steps=args.log_freq,
        save_total_limit=1,

        dataloader_drop_last=True,
        dataloader_num_workers=args.num_proc,

        local_rank=args.local_rank,
        deepspeed=args.deepspeed,
        fp16=args.fp16,
    )

In [14]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"],
    tokenizer = codet5p_tokenizer,
)

trainer.train()

/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dongchirua. Use `wandb login --relogin` to force relogin


/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,9.137700
10,9.034100
20,7.767100
30,6.492900
40,5.916000
50,5.572200
60,5.154400
70,4.718700
80,4.429400
90,4.238300


/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-package

TrainOutput(global_step=12100, training_loss=0.10344375216962061, metrics={'train_runtime': 35772.5036, 'train_samples_per_second': 13.58, 'train_steps_per_second': 0.338, 'total_flos': 6.90786607104e+17, 'train_loss': 0.10344375216962061, 'epoch': 99.79})

In [15]:
trainer.save_model(f'{args.save_dir}/{args.task}')

## measure on test set

In [16]:
sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=args.batch_size,
  num_workers=args.num_proc)

rouge_list = []

for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      max_length=args.max_des_length,
    ).cpu()
  labels = batch["labels"].cpu()

  rouge_result = metrics_func([preds, labels])
  print(rouge_result)
  rouge_list.append(rouge_result)

{'rouge1': 0.8588957055214724, 'rouge2': 0.6604938271604939, 'rougeL': 0.6993865030674846, 'rougeLsum': 0.8083832335329342}
{'rouge1': 0.8860759493670886, 'rouge2': 0.8177966101694916, 'rougeL': 0.8438818565400844, 'rougeLsum': 0.8838174273858922}
{'rouge1': 0.9318181818181819, 'rouge2': 0.8949771689497718, 'rougeL': 0.9090909090909091, 'rougeLsum': 0.9241071428571428}
{'rouge1': 0.8259385665529011, 'rouge2': 0.702054794520548, 'rougeL': 0.7303754266211603, 'rougeLsum': 0.8047138047138047}
{'rouge1': 0.9003984063745019, 'rouge2': 0.76, 'rougeL': 0.7848605577689242, 'rougeLsum': 0.8862745098039215}
{'rouge1': 0.8505747126436781, 'rouge2': 0.6615384615384615, 'rougeL': 0.685823754789272, 'rougeLsum': 0.8150943396226416}
{'rouge1': 0.8702594810379242, 'rouge2': 0.7374749498997996, 'rougeL': 0.7425149700598802, 'rougeLsum': 0.8487229862475442}
{'rouge1': 0.9537892791127541, 'rouge2': 0.8905380333951761, 'rougeL': 0.9057301293900184, 'rougeLsum': 0.9398907103825136}
{'rouge1': 0.92894280762

In [28]:
p = codet5p_tokenizer.batch_decode(preds[0], skip_special_tokens=True)
''.join(p).strip()

'v i a   a   c r a f t e d   a p p l i c a t i o n   t h a t   s e n d s   a   s i g n a l   t o   a   B l u e t o o t h   p r o c e s s'

In [26]:
l = codet5p_tokenizer.batch_decode(labels[0], skip_special_tokens=True)
''.join(l)

'b y   l e v e r a g i n g   t h e   a b i l i t y   t o   c r e a t e   a   r a w   s o c k e t  .'

In [19]:
rouge1_values = [d['rouge1'] for d in rouge_list]
rouge2_values = [d['rouge2'] for d in rouge_list]
rougeL_values = [d['rougeL'] for d in rouge_list]
rouge1_values = sum(rouge1_values) / len(rougeL_values)
average_rouge2 = sum(rouge2_values) / len(rougeL_values)
average_rougeL = sum(rougeL_values) / len(rougeL_values)

rouge1_values, average_rouge2, average_rougeL

(0.8799135777974783, 0.7595098570501858, 0.7668651879525156)